In [1]:
QTD_OF_SUMMARIES = 15
QTD_OF_TEST_SUMMARIES = 2
NR_EPOCHS = 100
BATCH_SIZE = 200

In [2]:
import datautils.loader.dataloader as loader

summaries = loader.load_anomalous_summaries()

In [3]:
import datautils.loader.dataloader as loader
loader.load_mne_data(summaries[:QTD_OF_SUMMARIES])
loader.load_time_segmented_data(summaries[:QTD_OF_SUMMARIES])

Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb01\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)


Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb01\chb01_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb01\chb01_15.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)
c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)


Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb01\chb01_16.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb01\chb01_18.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


...

c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)
c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)


Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb03\chb03_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb03\chb03_34.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)
c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)


Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb03\chb03_35.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)


### Data Preparation -> Dominio do Tempo 

- Segmentar as informações de tempo em janelas
- Criar as labels para cada janela
- Separar as informações em Treino e Teste
- Alimentar a rede e verificar a saída

In [4]:
import datautils.splitter.signalsplitter as splitter
X_train, X_val, y_train, y_val = splitter.segmented_time_data_splitter(summaries[:QTD_OF_SUMMARIES])

In [5]:
import datautils.balancer.databalancer as balancer
X_train, y_train = balancer.balance(X_train, y_train)
X_val, y_val = balancer.balance(X_val, y_val)

In [6]:
import ia.model.cnn as ia
time_cnn = ia.CNNModel(input_shape=(X_train[0].shape[0], X_train[0].shape[1]))

In [7]:
time_cnn.compile()

In [8]:
time_cnn.fit(X_train, y_train, num_epochs=NR_EPOCHS, batch_size=BATCH_SIZE, val_data=X_val, val_labels=y_val)

Epoch 1/100
2/2 [==============================] - 1s 168ms/step - loss: 0.6932 - accuracy: 0.5490 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/100
2/2 [==============================] - 0s 47ms/step - loss: 0.6932 - accuracy: 0.4804 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/100
2/2 [==============================] - 0s 35ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/100
2/2 [==============================] - 0s 41ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/100
2/2 [==============================] - 0s 41ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/100
2/2 [==============================] - 0s 38ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/100
2/2 [==============================] - 0s 35ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 8/100
2/2 [==

In [9]:
loader.load_mne_data(summaries[QTD_OF_SUMMARIES + 1 : QTD_OF_SUMMARIES + 1 + QTD_OF_TEST_SUMMARIES])
loader.load_time_segmented_data(summaries[QTD_OF_SUMMARIES + 1 : QTD_OF_SUMMARIES + 1 + QTD_OF_TEST_SUMMARIES])

Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb04\chb04_05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2441215  =      0.000 ...  9535.996 secs...


c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)


Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb04\chb04_08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3686399  =      0.000 ... 14399.996 secs...


c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)


In [13]:
import datautils.splitter.signalsplitter as splitter
X_test, _, y_test, _ = splitter.segmented_time_data_splitter(summaries[QTD_OF_SUMMARIES + 1 : QTD_OF_SUMMARIES + 1 + QTD_OF_TEST_SUMMARIES])

import datautils.balancer.databalancer as balancer
X_test, y_test = balancer.balance(X_train, y_train)


In [14]:
predictions = time_cnn.model.predict(X_test)
for i in range(len(y_test)):
    print(str(predictions[i] > 0.5) + ":" + str(y_test[i]))

10/10 [==============================] - 0s 2ms/step
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:False
[False]:F

In [15]:
test_loss, test_acc = time_cnn.model.evaluate(X_test, y_test)

print(f'Test accuracy: {test_acc}')

10/10 [==============================] - 0s 1ms/step - loss: 0.0587 - accuracy: 0.9935
Test accuracy: 0.9934640526771545
